<a href="https://colab.research.google.com/github/chiraz836/data-science/blob/main/first_part_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit 

     |████████████████████████████████| 10.1 MB 6.9 MB/s 
     |████████████████████████████████| 164 kB 52.8 MB/s 
     |████████████████████████████████| 4.3 MB 38.9 MB/s 
     |████████████████████████████████| 111 kB 53.9 MB/s 
     |████████████████████████████████| 181 kB 50.5 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 131 kB 38.5 MB/s 
     |████████████████████████████████| 130 kB 61.3 MB/s 
     |████████████████████████████████| 428 kB 35.5 MB/s 
     |████████████████████████████████| 793 kB 47.6 MB/s 
     |████████████████████████████████| 381 kB 47.0 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=31c9c74710c6c52eee9711530dddb61e0c5c91a860caf00b1282e4e9112b8e20
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
  Created wheel for validators: filename=validators

In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.891s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [3]:
%%writefile app.py
import numpy as np
import streamlit as st
import NoWhere as nh
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from math import sqrt, pow, exp
import spacy
nlp = spacy.load("en_core_web_sm")

st.title("Project Validation")

expander = st.expander("About this app", expanded=True)
with expander:
    st.info("""
    The **COURZELO** *course recom* app is an easy-to-use interface built in Streamlit using many similarity algorithm  !""")


dataset_btn = st.sidebar.button("Show Dataset")

text_input = st.sidebar.text_input("Search For A Course", key="1")
recnum = st.sidebar.slider("Pick the number of courses you want", min_value=1, max_value=50, value=20, step=1)


genre = st.sidebar.radio(
     "What's your favorite similarity measures ?",
     ('Cosine Similarity', 'Euclidean Distance'))

is_in_the_way = st.checkbox ("Show Recommendation Result ")

if is_in_the_way:
  if text_input:
    if genre == 'Cosine Similarity':
      cosres = nh.recommend_course(text_input, recnum)
      cosres= cosres.values
      cosres = cosres.tolist()
      st.write('## This is a simple implementation of `Cosine similarity` using `Courses Dataset`')
      for i in cosres:
        st.markdown(f'* {i[0]}')
    else:
      if genre == 'Euclidean Distance':
        cosres = nh.recommend_course_euc(text_input, recnum)
        cosres= cosres.values
        cosres = cosres.tolist()
        st.write('## This is a simple implementation of `Euclidean Distance` using `Courses Dataset`')
        for i in cosres:
          st.markdown(f'* {i[0]}')


jaccinp = []
jaccinp0 = st.sidebar.text_input("Enter fst jacc data", key="2")
jaccinp1 = st.sidebar.text_input("Enter scd jacc data", key="3")


X =jaccinp0.lower()
Y =jaccinp1.lower()
  
# tokenization
X_list = word_tokenize(X) 
Y_list = word_tokenize(Y)
  
# sw contains the list of stopwords
sw = stopwords.words('english') 
l1 =[];l2 =[]
  
# remove stop words from the string
X_set = {w for w in X_list if not w in sw} 
Y_set = {w for w in Y_list if not w in sw}
  

resultjacc = st.sidebar.button("Calculate Jaccard Distance")
resultcos = st.sidebar.button("Calculate Cosine similarity")
resulteuc = st.sidebar.button("Calculate Euclidean Distance")
jaccinp.append(jaccinp0)
jaccinp.append(jaccinp1)


if resultjacc:
  # text_input = False
  # genre = False
  jaccinp = [sent.lower().split(" ") for sent in jaccinp]
  jaccres = nh.jaccard_similarity(jaccinp[0], jaccinp[1])
  st.info(f'## the `similarity` between the two sentences using `Jaccard Distance` is : `{jaccres}` ')

if resultcos:
  # text_input = False
  # genre = False
  # form a set containing keywords of both strings 
  rvector = X_set.union(Y_set) 
  for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
  c = 0
  
  # cosine formula 
  for i in range(len(rvector)):
    c+= l1[i]*l2[i]
  cosine = c / float((sum(l1)*sum(l2))**0.5)
  st.info(f'## the `similarity` between the two sentences using `Cosine similarity` is : `{cosine}`')

if resulteuc:
  # text_input = False
  # genre = False
  jaccinp = [sent.lower().split(" ") for sent in jaccinp]
  embeddings = [nlp(str(sentence)).vector for sentence in jaccinp]
  distance = nh.euclidean_distance(embeddings[0], embeddings[1])
  st.info(f'## the `similarity` between the two sentences using `Euclidean Distance` is : `{nh.distance_to_similarity(distance)}`')


if dataset_btn:
  st.table(nh.fdf())


Writing app.py


In [7]:
%%writefile NoWhere.py
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel, euclidean_distances
from math import sqrt, pow, exp
import spacy
nlp = spacy.load("en_core_web_sm")


def squared_sum(x):
  """ return 3 rounded square rooted value """
 
  return round(sqrt(sum([a*a for a in x])),3)
 
def euclidean_distance(x,y):
  """ return euclidean distance between two lists """
 
  return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))


def distance_to_similarity(distance):
  return 1/exp(distance)


def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

df = pd.read_csv("/content/drive/MyDrive/Combined_Clean_Courses.csv")

def fdf():
  df1 = df.drop(['Unnamed: 0'], axis=1)
  return df1

count_vect = CountVectorizer()
cv_mat = count_vect.fit_transform(df['clean_course_title'].values.astype('U'))

df_cv_words = pd.DataFrame(cv_mat.todense(),columns=count_vect.get_feature_names())

cosine_sim_mat = cosine_similarity(cv_mat)
course_indices = pd.Series(df.index,index=df['clean_course_title']).drop_duplicates()


def recommend_course(title,num_of_rec=10):
    # ID for title
    idx = course_indices[title]
    # Course Indice
    # Search inside cosine_sim_mat
    scores = list(enumerate(cosine_sim_mat[idx]))
    # Scores
    # Sort Scores
    sorted_scores = sorted(scores,key=lambda x:x[1],reverse=True)
    # Recomm
    selected_course_indices = [i[0] for i in sorted_scores[1:]]
    selected_course_scores = [i[1] for i in sorted_scores[1:]]
    result = df['clean_course_title'].iloc[selected_course_indices]
    rec_df = pd.DataFrame(result)
    rec_df['similarity_scores'] = selected_course_scores
    return rec_df.head(num_of_rec)



euclidean_distances_mat = euclidean_distances(cv_mat)
course_indices_euc = pd.Series(df.index,index=df['clean_course_title']).drop_duplicates()


def recommend_course_euc(title,num_of_rec=10):
    # ID for title
    idx = course_indices[title]
    # Course Indice
    # Search inside cosine_sim_mat
    scores = list(enumerate(euclidean_distances_mat[idx]))
    # Scores
    # Sort Scores
    sorted_scores = sorted(scores,key=lambda x:x[1],reverse=True)
    # Recomm
    selected_course_indices = [i[0] for i in sorted_scores[1:]]
    selected_course_scores = [i[1] for i in sorted_scores[1:]]
    result = df['clean_course_title'].iloc[selected_course_indices]
    rec_df = pd.DataFrame(result)
    rec_df['similarity_scores'] = selected_course_scores
    return rec_df.head(num_of_rec)


Overwriting NoWhere.py


In [8]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-11 23:28:34.503 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.855s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.251.86:8501

your url is: https://purple-years-fetch-35-245-251-86.loca.lt
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Pac

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
